In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Web Dash
from flask import Flask
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import datetime

from abides_core import abides
from abides_core.utils import parse_logs_df, ns_date, str_to_ns, fmt_ts
from abides_markets.configs import rmsc05

config = rmsc05.build_config(
    end_time="10:00:00"
)


In [ ]:
config.keys()
end_state = abides.run(config)


In [ ]:
# Ex_0_L3_ITCH = Ex_0_order_book.get_l3_itch()
# Ex0_best_bids = pd.DataFrame(Ex_0_L1["best_bids"],columns=["time","price","qty"])
# Ex0_best_asks = pd.DataFrame(Ex_0_L1["best_asks"],columns=["time","price","qty"])
# Ex0_best_bids['price'] = Ex0_best_bids['price'].div(100)
# Ex0_best_asks['price'] = Ex0_best_asks['price'].div(100)

# Ex_1_order_book = end_state["agents"][1].order_books["ABM"]
# Ex_1_L1 = Ex_1_order_book.get_L1_snapshots()
# Ex1_best_bids = pd.DataFrame(Ex_1_L1["best_bids"],columns=["time","price","qty"])
# Ex1_best_asks = pd.DataFrame(Ex_1_L1["best_asks"],columns=["time","price","qty"])
# Ex1_best_bids['price'] = Ex1_best_bids['price'].div(100)
# Ex1_best_asks['price'] = Ex1_best_asks['price'].div(100)

In [ ]:
Ex_0_order_book = end_state["agents"][0].order_books["ABM"]
ex_0_ob_imbalance = Ex_0_order_book.get_imbalance()
ex_0_name = Ex_0_order_book.owner.name

In [ ]:
Ex_1_order_book = end_state["agents"][1].order_books["ABM"]
ex_1_ob_imbalance = Ex_1_order_book.get_imbalance()
ex_1_name = Ex_1_order_book.owner.name

In [ ]:
logs_df = parse_logs_df( end_state )

In [ ]:
logs_df.to_csv("logs_df.csv")

In [ ]:
executed_orders =  logs_df[logs_df.EventType=="ORDER_EXECUTED"]
executed_orders = executed_orders.sort_values(by='EventTime', ascending=True)


In [ ]:
ex_0_submitted_orders=executed_orders.loc[(executed_orders['exchange_id'] == 0.0)]
ex_1_submitted_orders=executed_orders.loc[(executed_orders['exchange_id'] == 1.0)]

In [ ]:
# create new column with only value 1 for each row
ex_0_submitted_orders['count'] = 1
ex_1_submitted_orders['count'] = 1

ex_0_submitted_orders['cumsum_order_qty'] = ex_0_submitted_orders['count'].cumsum()
ex_1_submitted_orders['cumsum_order_qty'] = ex_1_submitted_orders['count'].cumsum()

ex_0_submitted_orders['cumsum_qty'] = ex_0_submitted_orders['quantity'].cumsum()
ex_1_submitted_orders['cumsum_qty'] = ex_1_submitted_orders['quantity'].cumsum()

ex_0_submitted_orders.drop(columns=['count'], inplace=True)
ex_1_submitted_orders.drop(columns=['count'], inplace=True)



In [ ]:
import plotly.graph_objects as go

executed_order_qty = go.Figure()
executed_order_volume = go.Figure()
executed_order_qty.add_trace(go.Scatter(x=ex_0_submitted_orders.EventTime, y=ex_0_submitted_orders["cumsum_order_qty"], mode='lines', name='Exchange 0 - Executed Order Quantity'))
executed_order_qty.add_trace(go.Scatter(x=ex_1_submitted_orders.EventTime, y=ex_1_submitted_orders["cumsum_order_qty"], mode='lines', name='Exchange 1 - Executed Order Quantity'))
executed_order_volume.add_trace(go.Scatter(x=ex_0_submitted_orders.EventTime, y=ex_0_submitted_orders["cumsum_qty"], mode='lines', name='Exchange 0 - Executed Order Volumes'))
executed_order_volume.add_trace(go.Scatter(x=ex_1_submitted_orders.EventTime, y=ex_1_submitted_orders["cumsum_qty"], mode='lines', name='Exchange 1 - Executed Order Volumes'))



